In [1]:
import tensorflow as tf
# Fixes bad convolution
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [108]:
def bin2num(binary, bits=36):
    exponents = 2**tf.range(bits-1, -1, -1, tf.int64)
    return tf.reduce_sum(binary*exponents)

s = tf.io.read_file('day14.txt')

@tf.function
def solve1(s):
    s = tf.strings.split(s, '\n')
    s = tf.strings.split(s, ' = ')
    s = s.to_tensor()
    n = tf.shape(s)[0]

    tokens, idxs = tf.unique(s[:,0])
    # index 0 is the mask row, this makes it easy to filter and work with later
    idxs -= 1

    ta = tf.TensorArray(tf.int64, size=tf.shape(tokens)[0]-1)

    keep_val = tf.constant(0, tf.int64)
    change_val = tf.constant(0, tf.int64)
    
    for i in tf.range(n):
        line = s[i]
        if line[0] == 'mask':
            mask = tf.strings.bytes_split(line[1], 'UTF-8')
            change_bits = tf.scatter_nd(tf.where(mask != 'X'), tf.strings.to_number(mask[mask != 'X'], out_type=tf.int64), (36,))
            keep_bits = tf.where(mask == 'X', tf.constant(1, tf.int64), tf.constant(0, tf.int64))

            change_val = bin2num(change_bits)
            keep_val = bin2num(keep_bits)
        else:
            num = tf.strings.to_number(line[1], tf.int64)
            num = (num&keep_val)|change_val
            ta = ta.write(idxs[i], num)

    return tf.reduce_sum(ta.stack())

%timeit solve(s)
solve(s)

139 ms ± 3.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<tf.Tensor: shape=(), dtype=int64, numpy=14553106347726>

In [494]:
@tf.function(experimental_compile=True)
def bin2num(binary, bits=36):
    exponents = 2**tf.range(bits-1, -1, -1, tf.int64)
    return tf.reduce_sum(binary*exponents)

@tf.function(experimental_compile=True)
def num2bin(num, n):
    return (tf.bitwise.right_shift(tf.expand_dims(num, 1), tf.range(n, dtype=tf.int64)) % tf.constant(2, tf.int64))[:,::-1]

s = tf.io.read_file('day14.txt')

@tf.function
def solve2(s):
    s = tf.strings.regex_replace(s, 'mem\[', '')
    s = tf.strings.regex_replace(s, '\]', '')
    s = tf.strings.split(s, '\n')
    s = tf.strings.split(s, ' = ')
    s = s.to_tensor()
    n = tf.shape(s)[0]

    ta = tf.TensorArray(tf.int64, size=0, dynamic_size=True)
    ta_idx = 0

    change_val = tf.constant(0, tf.int64)
    floating_val = tf.constant(0, tf.int64)

    i = 0

    while i < n:
        line = s[i]
        if line[0] == 'mask':
            mask = tf.strings.bytes_split(line[1], 'UTF-8')
            change_bits = tf.scatter_nd(tf.where(mask != 'X'), tf.strings.to_number(mask[mask != 'X'], out_type=tf.int64), (36,))
            X_mask = mask == 'X'
            floating_bits = tf.where(X_mask, tf.constant(1, tf.int64), tf.constant(0, tf.int64))
            n_bits = tf.math.count_nonzero(X_mask)
            floating_indices = tf.where(floating_bits == 1)
            non_floating_bits = 1 - floating_bits

            floating_val = bin2num(floating_bits)
            change_val = bin2num(change_bits)
            non_floating_val = bin2num(non_floating_bits)
            
            binary_vals = num2bin(tf.range(2**n_bits, dtype=tf.int64), n_bits)
            binary_vecs = tf.map_fn(
                lambda bv: bin2num(tf.scatter_nd(floating_indices, bv, (36,))),
                binary_vals
            )
            
            i += 1
            while i < n and s[i][0] != 'mask':
                line = s[i]
                address = tf.strings.to_number(line[0], tf.int64)
                num = tf.strings.to_number(line[1], tf.int64)

                addresses = tf.map_fn(
                    lambda bv: ((address|change_val)&non_floating_val)|bv,
                    binary_vecs
                )
                
                for current_address in addresses:
                    ta = ta.write(ta_idx, (current_address, num))
                    ta_idx += 1

                i += 1

    res = ta.stack()

    addresses, idxs = tf.unique(res[::-1,0])

    return tf.reduce_sum(tf.map_fn(
        lambda i: res[::-1,1][tf.where(idxs == i)[0][0]],
        tf.range(tf.shape(addresses)[0]),
        fn_output_signature=tf.TensorSpec(None, tf.int64)
    ))

In [ ]:
%%time
solve2(s)

0
6
11
16
20
24
30
36
42
46
51
55
63
67
70
76
82
86
91
96
101
108
111
116
123
127
132
140
143
146
151
157
164
168
175
182
188
191
196
204
212
218
226
230
234
240
246
